In [1]:
import os
from dotenv import load_dotenv
import openai
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType, Vector

load_dotenv()
# Replace these with your own values, either in environment variables or directly here
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT")
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER")
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX") 
AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE") 
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT") 
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT") 
AZURE_OPENAI_EMB_DEPLOYMENT = os.environ.get("AZURE_OPENAI_EMB_DEPLOYMENT")
AZURE_SEARCH_SERVICE_KEY = os.environ.get("AZURE_SEARCH_SERVICE_KEY") 

KB_FIELDS_CONTENT = os.environ.get("KB_FIELDS_CONTENT") or "content"
KB_FIELDS_CATEGORY = os.environ.get("KB_FIELDS_CATEGORY") or "category"
KB_FIELDS_SOURCEPAGE = os.environ.get("KB_FIELDS_SOURCEPAGE") or "sourcepage"


# Used by the OpenAI SDK
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-06-01-preview"
openai.api_key = os.environ.get("AZURE_OPENAI_API_KEY") 
# Comment these two lines out if using keys, set your API key in the OPENAI_API_KEY environment variable and set openai.api_type = "azure" instead
openai.api_type = "azure"

# Set up clients for Cognitive Search and Storage
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(AZURE_SEARCH_SERVICE_KEY))

In [3]:
# ChatGPT uses a particular set of tokens to indicate turns in conversations
prompt_prefix = """<|im_start|>system
어시스턴트는 검색 결과에서 나온 문서에 대한 질문을 도와줍니다. 답변은 간결하게 작성하세요.
검색결과에 나오지 않은 내용은 답변하지 않습니다. 검색결과에 나온 내용을 잘 읽어보세요.
표 형식의 정보는 HTML 테이블로 반환합니다. 마크다운 형식은 반환하지 마세요.
각 출처에는 이름 뒤에 콜론과 실제 정보가 있으며, 응답에 사용하는 각 사실에 대한 출처 이름을 항상 포함하세요. 소스를 참조할 때는 대괄호를 사용합니다(예: [info1.txt]). 소스를 결합하지 말고 각 소스를 개별적으로 나열합니다(예: [info1.txt][info2.pdf]).
Sources:
{sources}

<|im_end|>"""

turn_prefix = """
<|im_start|>user
"""

turn_suffix = """
<|im_end|>
<|im_start|>assistant
"""

prompt_history = turn_prefix

history = []

summary_prompt_template = """아래는 지금까지의 대화 요약과 사용자가 검색결과에서 검색하여 답변해야 하는 새로운 질문입니다. 대화와 새 질문을 기반으로 검색 쿼리를 생성합니다.

Summary:
{summary}

Question:
{question}

Search query:
"""

In [4]:
# Execute this cell multiple times updating user_input to accumulate chat history
user_input = "인터넷 형광펜 서비스 소개좀 부탁해요"

# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

if len(history) > 0:
    completion = openai.Completion.create(
        engine=AZURE_OPENAI_GPT_DEPLOYMENT,
        prompt=summary_prompt_template.format(summary="\n".join(history), question=user_input),
        temperature=1,
        max_tokens=32,
        stop=["\n"])
    search = completion.choices[0].text
else:
    search = user_input

# Use Azure OpenAI to compute an embedding for the query
query_vector = openai.Embedding.create(engine=AZURE_OPENAI_EMB_DEPLOYMENT, input=search)["data"][0]["embedding"]


print("Searching:", search)
print("-------------------")
filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None

# Hybrid search with semantic search and vector reranking

r = search_client.search(search, 
                         filter=filter,
                         query_type=QueryType.SEMANTIC, 
                         query_language="en-us", 
                         query_speller="lexicon", 
                         semantic_configuration_name="default", 
                         top=3,
                         vector=Vector(value=query_vector, k=50, fields="embedding") if query_vector else None)

# Vector search only
# r = search_client.search(search, top=3, vector=Vector(value=query_vector, k=50, fields="embedding") if query_vector else None)

results = [doc[KB_FIELDS_SOURCEPAGE] + ": " + doc[KB_FIELDS_CONTENT].replace("\n", "").replace("\r", "") for doc in r]
content = "\n".join(results)

prompt = prompt_prefix.format(sources=content) + prompt_history + user_input + turn_suffix

print("\n-------------------\n".join(history))
print("\n-------------------\nPrompt:\n" + prompt)

completion = openai.Completion.create(
    engine=AZURE_OPENAI_CHATGPT_DEPLOYMENT, 
    prompt=prompt, 
    temperature=0.7, 
    max_tokens=1024,
    stop=["<|im_end|>", "<|im_start|>"])

prompt_history += user_input + turn_suffix + completion.choices[0].text + "\n<|im_end|>" + turn_prefix
history.append("user: " + user_input)
history.append("assistant: " + completion.choices[0].text)

print("\n-------------------\n".join(history))




Searching: 인터넷 형광펜 서비스 소개좀 부탁해요
-------------------


-------------------
Prompt:
<|im_start|>system
어시스턴트는 검색 결과에서 나온 문서에 대한 질문을 도와줍니다. 답변은 간결하게 작성하세요.
검색결과에 나오지 않은 내용은 답변하지 않습니다. 검색결과에 나온 내용을 잘 읽어보세요.
표 형식의 정보는 HTML 테이블로 반환합니다. 마크다운 형식은 반환하지 마세요.
각 출처에는 이름 뒤에 콜론과 실제 정보가 있으며, 응답에 사용하는 각 사실에 대한 출처 이름을 항상 포함하세요. 소스를 참조할 때는 대괄호를 사용합니다(예: [info1.txt]). 소스를 결합하지 말고 각 소스를 개별적으로 나열합니다(예: [info1.txt][info2.pdf]).
Sources:
dbr_sample-2.pdf:  압축 성장을 이루는 동시에 독자적인 비즈니스 모델을 갖추기까지 어떤 시행착오를 거쳤을까. DBR(동아비즈니스리뷰)이 아우름플래닛을 창업하고 현재는 미국 법인 LINER Inc.의 대표를 맡고 있는 김진우 대표를 만나 경쟁력 소프트 엣지’ 등 브라우저에서 라이너 확장 프로그램을 설치하고 구글, 네이버 등에서 검색을 하면 사용자와 관련성 높은 검색 결과가 큐레이션 된다. 올해 4월에는 GPT-4를 적용한 인공지능(AI) 검색용 챗봇 ‘라이너챗’을 출시했다. 최신 정보까지 소스로 활용하고 사용자 맞춤으로 답변을 제공한다. 라이너 웹 사이트나 앱에 접속하지 않아도 검색 포털과 연동돼 바로 활용할 수 있다는 점도 편리하다. 검색 결과 페이지에서 텍스트를 드래그하면 해당 내용을 저장할 수도 있고 번역, 추가 검색도 바로 할 수 있다. 라이너 앱이나 웹 사이트에서는 자체적인 검색 페이지와 라이너챗, 추천 콘텐츠 피드를 제공한다. 라이너가 초개인화된 추천 서비스를 제공할 수 있는 원천은 사용자들의 ‘형광펜’ 데이터다. 사실 라이너 자체가 인터넷에서 활용 가능한 형광펜 서비스로 시작했다. 책에서 중요한 부분

In [13]:
history.clear()